In [1]:
import PyPDF2
import urllib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import io
import textract
import os

In [2]:
#scrapes the PTAB website to find all recent decisions and metadata

def gen_valid_dates():
    d_to_get = []
    for yr in range(2015, 2021):
        syr = str(yr)
        for month in range(1, 13):
            smth = str(month).zfill(2)
            d_to_get.append(syr + '_' + smth)
    return d_to_get[0:9] + d_to_get[10:62] + [d_to_get[63]]

def process_soup(soup, date):
    trs = soup.find_all('table')[1].find_all('tr')
    results = []
    for i in range(1, len(trs)):
        cur_tr = trs[i]    
        tds = cur_tr.find_all("td")
        sub_dict = {}
        sub_dict['link'] = tds[0].find('a')['href']
        sub_dict['appellant'] = tds[1].text
        sub_dict['county'] = tds[2].text
        sub_dict['reason_code'] = tds[3].text
        docket = tds[0].text.split('-')
        sub_dict['prop_type'] = docket[2]
        sub_dict['valuation_class'] = docket[3]
        sub_dict['docket_name'] = docket[0] + '-' + docket[1]
        sub_dict['date'] = date
        results.append(sub_dict)
    return pd.DataFrame(results)

def get_itemized_decisions():
    valid_dates = gen_valid_dates()
    for i, date in enumerate(valid_dates):
        print(date)
        response = requests.get('http://www.ptab.illinois.gov/decisions/default.asp?sdf=' + date)
        time.sleep(5)
        soup = BeautifulSoup(response.content, 'html.parser')
        if i == 0:
            result = process_soup(soup, date)
        else:
            result = result.append(process_soup(soup, date))
    return result

In [3]:
decisions = get_itemized_decisions()
decisions.to_csv('decisions.csv', index=False)
decisions = pd.read_csv('decisions.csv')

In [1]:
#downloads pdfs and extracts text

def link_to_text_old(link):
    response = requests.get(link, timeout=15) 
    with io.BytesIO(response.content) as f:
        pdf = PyPDF2.PdfFileReader(f)
        num_page = pdf.getNumPages()
        print(num_page)
        full_str = ''
        for i in range(num_page):
            page = pdf.getPage(i)
            text = page.extractText()
            full_str = full_str + text
    return full_str

def link_to_text(link):
    tmp_loc = 'C:/Users/erhla/Documents/GitHub/ptab_project/tmp.pdf'
    response = requests.get(link, timeout=15) 
    open('C:/Users/erhla/Documents/GitHub/ptab_project/tmp.pdf', 'wb').write(response.content)
    text = textract.process(tmp_loc)
    return text.decode("utf-8") 

def process_entry(entry):
    try:
        text = link_to_text(entry['link'])
        f = open('files/' + entry['docket_name'] + '.txt', "w+")
        f.write(text)
        f.close()
        #entry.to_frame().T.to_csv('success.csv', mode='a', header=False, index=False)
    except:
        print(entry['docket_name'])
        #entry.to_frame().T.to_csv('failed.csv', mode='a', header=False, index=False)

def process_decisions(decisions):
    for index, row in decisions.iterrows():
        if index % 100 == 0:
            print(index)
        process_entry(row)

process_decisions(decisions)

In [15]:
good_files = set([i[:10] for i in os.listdir('files')])
total_files = set(decisions.docket_name.tolist())
files_to_run = total_files.difference(good_files)
len(files_to_run)

8

In [16]:
process_decisions(decisions[decisions.docket_name.isin(files_to_run)])

# these decisions are missing but listed as issued

2013-34567
2015-23709
2015-30475
2016-21187
2016-21312
2014-03642
2014-03653
2017-44543
